# Vaults - Setup

## Implementation Notes
- Call totalAssets, totalCollectableFees, and totalSupply from 0x221B2D9aD7B994861Af3f4c8A80c86C4aa86Bf53 via web3.py
- Get transfers from subgraph

In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from mezo.currency_utils import get_token_price, format_musd_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [3]:
# Subgraph Setup

AUGUST_VAULT_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/mezo-vaults-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_VAULT_TRANSFERS = """
query vaultTransfers($skip: Int!) {
  transfers(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    from
    to
    value
    transactionHash_
    block_number
  }
}
"""

In [4]:
# Instantiate subgraph

vault = SubgraphClient(
    url = AUGUST_VAULT_SUBGRAPH, 
    headers = SUBGRAPH_HEADERS
)

In [5]:
# Fetch subgraph data
vaults =  vault.fetch_subgraph_data(GET_VAULT_TRANSFERS, 'transfers')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
No more records found.


In [6]:
# Save vaults data df

vaults_df = pd.DataFrame(vaults)
display(vaults_df)

,timestamp_,from,to,value,transactionHash_,block_number
0,1753805802,0x0000000000000000000000000000000000000000,0xae16dc30377bfbd511a7ac0ddbda2cf9c75bc061,437904309258229145,0x7354ca8d3ec3b7663975611a32e672d85e4087fa39a6...,2053277
1,1753805791,0x0000000000000000000000000000000000000000,0x55d07f4e9b54c911ed6001bd7ec1ba0a9a8b3c97,432559512605627176,0xd4b79acb5dd6a335691c73588f6f37134d96ff9c8a4d...,2053274
2,1753805765,0x0000000000000000000000000000000000000000,0xa545317b81f00e95528812d5779c921931a4f028,531194662480384644,0xfbc93d786fca942d247930a58197b7e1f9cb964f7366...,2053267
3,1753805416,0x0000000000000000000000000000000000000000,0x81b42b3e38bd759bc80d9997f6cd2118a30933cd,167547770559767849,0xb0916f9b319da546e6a6f312d86b707cb64f15591cef...,2053170
4,1753804687,0x0000000000000000000000000000000000000000,0xdd28492a83b3895dd480495eff36423afdc7ed4c,1000000000000000000,0xfc053725bc1a53eaa3c9260190ab4277e345acbe5890...,2052967
...,...,...,...,...,...,...
3080,1750924920,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1000000000000000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
3081,1750899066,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5000000000000000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
3082,1750894224,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10000000000000000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
3083,1750853682,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1000000000000000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [7]:
# Exploration 
format_musd_currency_columns(vaults_df, ['value'])

,timestamp_,from,to,value,transactionHash_,block_number
0,1753805802,0x0000000000000000000000000000000000000000,0xae16dc30377bfbd511a7ac0ddbda2cf9c75bc061,0.437904,0x7354ca8d3ec3b7663975611a32e672d85e4087fa39a6...,2053277
1,1753805791,0x0000000000000000000000000000000000000000,0x55d07f4e9b54c911ed6001bd7ec1ba0a9a8b3c97,0.432560,0xd4b79acb5dd6a335691c73588f6f37134d96ff9c8a4d...,2053274
2,1753805765,0x0000000000000000000000000000000000000000,0xa545317b81f00e95528812d5779c921931a4f028,0.531195,0xfbc93d786fca942d247930a58197b7e1f9cb964f7366...,2053267
3,1753805416,0x0000000000000000000000000000000000000000,0x81b42b3e38bd759bc80d9997f6cd2118a30933cd,0.167548,0xb0916f9b319da546e6a6f312d86b707cb64f15591cef...,2053170
4,1753804687,0x0000000000000000000000000000000000000000,0xdd28492a83b3895dd480495eff36423afdc7ed4c,1.000000,0xfc053725bc1a53eaa3c9260190ab4277e345acbe5890...,2052967
...,...,...,...,...,...,...
3080,1750924920,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
3081,1750899066,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5.000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
3082,1750894224,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10.000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
3083,1750853682,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [8]:
format_datetimes(vaults_df, ['timestamp_'])

,timestamp_,from,to,value,transactionHash_,block_number
0,2025-07-29,0x0000000000000000000000000000000000000000,0xae16dc30377bfbd511a7ac0ddbda2cf9c75bc061,0.437904,0x7354ca8d3ec3b7663975611a32e672d85e4087fa39a6...,2053277
1,2025-07-29,0x0000000000000000000000000000000000000000,0x55d07f4e9b54c911ed6001bd7ec1ba0a9a8b3c97,0.432560,0xd4b79acb5dd6a335691c73588f6f37134d96ff9c8a4d...,2053274
2,2025-07-29,0x0000000000000000000000000000000000000000,0xa545317b81f00e95528812d5779c921931a4f028,0.531195,0xfbc93d786fca942d247930a58197b7e1f9cb964f7366...,2053267
3,2025-07-29,0x0000000000000000000000000000000000000000,0x81b42b3e38bd759bc80d9997f6cd2118a30933cd,0.167548,0xb0916f9b319da546e6a6f312d86b707cb64f15591cef...,2053170
4,2025-07-29,0x0000000000000000000000000000000000000000,0xdd28492a83b3895dd480495eff36423afdc7ed4c,1.000000,0xfc053725bc1a53eaa3c9260190ab4277e345acbe5890...,2052967
...,...,...,...,...,...,...
3080,2025-06-26,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xf94447d87f136b744db7d595d441d468fca44162c337...,1257108
3081,2025-06-26,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,5.000000,0x6a633ddd97ece5fd9c72271c72745a20c75635f93932...,1250310
3082,2025-06-25,0x0000000000000000000000000000000000000000,0x83db892a6688b966122be219fa873bf24d7db79f,10.000000,0xb1414979063a26c67326912e08202b38a096e43456aa...,1249036
3083,2025-06-25,0x0000000000000000000000000000000000000000,0x6e80164ea60673d64d5d6228beb684a1274bb017,1.000000,0xeadd15fb5deaf6f11e2bfd1a9421ae40de826f8a5e3e...,1238184


In [15]:
vaults_df['transactionHash_'][0]


'0x7354ca8d3ec3b7663975611a32e672d85e4087fa39a6fe6e7b993751cfa69dfc'

In [9]:
tvl = sum(vaults_df['value'])

In [10]:
depositors = pd.DataFrame(vaults_df['to'].unique()).count()
deposits = vaults_df['to'].count()

In [11]:
deposits

np.int64(3085)

In [10]:
vaults_df.dtypes

timestamp_           object
from                 object
to                   object
value               float64
transactionHash_     object
block_number         object
dtype: object

In [11]:
vaults_df['count'] = 1

In [17]:
daily_vault_txns = vaults_df.groupby(['timestamp_']).agg(
    value = ('value', 'sum'),
    transactions = ('count', 'sum'),
    users = ('to', lambda x: x.nunique())
).reset_index().sort_values(by='timestamp_', ascending=False)

In [26]:
daily_vault_txns

,timestamp_,value,transactions,users
4,2025-07-17,31456.294951,21,21
3,2025-07-16,87524.961135,42,32
2,2025-07-14,1.046000,1,1
1,2025-06-26,10.100223,9,4
0,2025-06-25,12.000000,3,2


In [3]:
# instantiate web3 client

w3 = Web3Client('AugustVault')
august_vault = w3.load_contract()
august_vault.all_functions()

FileNotFoundError: [Errno 2] No such file or directory: 'mezo/smart_contracts/AugustVault.json'

In [ ]:
# instantiate bigquery
bq = BigQueryClient(key='GOOGLE_CLOUD_KEY', project_id='mezo-portal-data')

In [ ]:
GET_POOL_FEES = """
query poolFees ($skip: Int!) {
  fees(
    orderBy: timestamp_
    orderDirection: desc
    first: 1000
    skip: $skip
) {
    timestamp_
    sender
    amount0
    amount1
    contractId_
    transactionHash_
  }
}
"""